In [1]:
from src.decode import read_source, parse_program, dump_file
from src.funcs import *
from src.analyze import sorted_defs, get_unique_fns, get_unique_refs, get_all_unique_fns, is_token, inline

In [2]:
defs = parse_program(read_source())

In [3]:
AP(CAR, AP(CDR, AP(CDR, AP(CDR, AP(AP(CONS, 1), AP(AP(CONS, 2), AP(AP(CONS, 3), AP(AP(CONS, 4), NIL))))))))

4

In [4]:
get_all_unique_fns(defs)

{'add',
 'ap',
 'b',
 'c',
 'car',
 'cdr',
 'cons',
 'div',
 'eq',
 'i',
 'isnil',
 'lt',
 'mul',
 'neg',
 'nil',
 's',
 't'}

In [11]:
get_unique_refs(inline(defs, 'galaxy', 2))

{':1328', ':1336', ':1342'}